# Analyse van de valbeweging

In deze notebook proberen we een wiskundig model op te stellen voor de valbeweging. Hiervoor gebruiken we de data die jij zelf hebt verzameld met behulp van de Dwenguino. Een wiskundig model voor een beweging is nuttig omdat we het kunnen gebruiken om andere valbewegingen te simuleren en zo voorspellingen te doen.

## Imorteren van bibliotheken
We starten met he importeren van de numpy bibliotheek. Deze bibliotheek maakt het gemakkelijk om gegevens te verwerken. Verder importeren we ook de matplitlib bibliotheek om gegevens te plotten.

In [ ]:
# Importeer de numpy bibliotheek als np
import numpy as np
import matplotlib.pyplot as plt

## Inlezen van de gegevens

Voor we kunnen starten met de analyse, moeten we de gegevens uit ons .csv bestand inladen in Python. Daarvoor gebruiken we de *lees_csv_bestand* functie. 

In [ ]:
def lees_csv_bestand(file_name):
    data = np.genfromtxt(file_name, skip_header=True, delimiter=';')
    return data

We roepen onze functie op met de padnaam van het bestand dat we willen inladen.

In [ ]:
gegevens_valbeweging = lees_csv_bestand('data/valbeweging1.csv')

De functie *lees_csv_bestand* zal de gegevens opslaan in een matrix. Elke rij van de matrix komt overeen met één meting. De eerste kolom bevat de tijden waarop de meting uitgevoerd is (in ms). De tweede kolom bevat de afstandsmeting zelf (in ms). Onderstaande code drukt de eerste vijf rijen van de matrix af.

In [ ]:
print(gegevens_valbeweging[0:5])

## De gegevens proper maken

In de uitvoer van de cel hierboven zie je dat zowel de tijd als de afstand vreemde waarden hebben. Bijgevolg gaan we de gegevens eerst wat opkuisen voor we die verder verwerken.

### Tijdstippen

 De tijd wordt weergegeven in miliseconden (= 1/1000 van een seconde) en de eerste meting start niet op tijdstip 0. We willen graag dat de tijd weergegeven wordt in seconden en dat de eerste meting start op tijdstip 0. Dat kunnen we doen met onderstaande code.

In [ ]:
# De eerste kolom bevat de tijdstippen in milliseconden, sla deze apart op in een variabele tijdstippen
tijdstippen = gegevens_valbeweging[:, 0]

# Trek van elk tijdstip het eerste tijdstip af. Zo starten de tijdstippen bij 0.
tijdstippen = tijdstippen - tijdstippen[0]

# Deel de tijdstippen door 1000 om ze om te zetten naar seconden.
tijdstippen = tijdstippen/1000 

We kunnen de aangepaste tijdstippen nu ook eens bekijken.

In [ ]:
print(tijdstippen[0:5])

### Afstanden

De afstanden worden in ons .csv bestand voorgesteld door tijden (in µs = 1/1000000 seconden). Deze waarden zijn de tijd die het geluid nodig had om vanop de sensor te vertrekken, tegen het object te botsen en terug te keren naar de sensor. Om deze tijden om te kunnen zetten naar afstanden moeten we dus weten wat de snelheid van het geluid is. De snelheid van het geluid in lucht is ongeveer 1cm/28.5µs. Als we een afstand van 1cm meten dan moet het geluid heen en terug gaan dus de tijd die verlopen zal zijn om 1cm te meten is 57µs. We gebruiken deze waarde om onze metingen om te zetten naar centimeter.

In [ ]:
# Zet de metingen om naar cm
US_PER_CM_HEEN_EN_TERUG = 57.0
gegevens_valbeweging[:, 1] = gegevens_valbeweging[:, 1] / US_PER_CM_HEEN_EN_TERUG

Omdat de sensor de afstand tot het vallende object van bovenaf meet en niet de hoogte van het object, moeten we de metingen omdraaien. Daarvoor vervangen we elke waarde door het verschil tussen de laatst gemeten waarde (de afstand tot de grond) en de gemeten waarde op dat moment. Zo krijgen we de hoogte van het object op elk tijdstip.

Eens we de afstand hebben in cm, kunnen we die ook eenvoudig omzetten naar meter.

In [ ]:
# Neem het verschil van de laaste meting en alle metingen. 
gegevens_valbeweging[:, 1] = gegevens_valbeweging[-1, 1] - gegevens_valbeweging[:, 1]
# Sla de afstanden op in een aparte variabele afstanden en zet deze om naar meter.
afstanden = gegevens_valbeweging[:, 1].reshape(-1, 1)
afstanden = afstanden / 100 

In [ ]:
print(afstanden[0:5])

## De beweging visualiseren

Nu we onze data proper gemaakt hebben, kunnen we die visualiseren. Onderstaande functie krijgt de gegevens, de tijdstippen en een titel mee als parameters en toont deze in een grafiek.

In [ ]:
# set plot size
plt.rcParams["figure.figsize"] = (20,10)
def plot_gegevens(gegevens, tijdstippen, titel='', xlabel="Tijd (s)", ylabel="Hoogte (m)", indices=[0]):
    plt.xticks(np.arange(0, len(gegevens), 10, dtype=np.uint32), tijdstippen[::10].round(2))
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    for index in indices:
        plt.plot(gegevens[:, index], label='y')
    plt.title(titel)
    plt.legend()
    plt.show()

We kunnen onze functie nu gebruiken om onze gegevens te visualiseren.

In [ ]:
plot_gegevens(afstanden, tijdstippen, titel='Verloop valbeweging', xlabel='Tijd (s)', ylabel='Hoogte (m)')

We zien dat de valbeweging zelf plaatsvindt tussen twee en 3 seconden. We kunnen de beweging dus beter analyseren wanneer we inzoomen op dat tijdsinterval.

In [ ]:
# Zoom in op de data tussen 2 en 3 seconden
start = 2
einde = 3
indices = np.where((tijdstippen > start) & (tijdstippen < einde))
plot_gegevens(afstanden[indices], tijdstippen[indices], titel='Verloop valbeweging', xlabel='Tijd (s)', ylabel='Hoogte (m)')

**Opdracht:** Herken je de curve? Volgens welk voorschrift zou de positie van het voorwerp veranderen?

## Een wiskundig model fitten

Door onze kennis over computerwetenschappen, fysica en wiskunde te combineren kunnen we een model maken van de beweging. Hiervoor baseren we ons op de data van onze sonar-sensor. 
Omdat we enkel een model willen van de valbeweging en niet van de momenten dat je het voorwerp vast had of wanneer het op de grond lag, zoomen we nog wat meer in op onze data.

In [ ]:
# We zoomen nu in op de data tussen 2.35 en 2.75 seconden
# Leg het startpunt en eindpunt vast
start = 2.350
einde = 2.750

# Zoek de indices van de tijdstippen die tussen start en einde liggen
indices = np.where((tijdstippen > start) & (tijdstippen < einde))

# Selecteer de afstanden en tijden die tussen start en einde liggen
tijden = tijdstippen[indices]
afstanden = afstanden[indices]

# Zorg dat de tijden starten bij 0
tijden = tijden - tijden[0]

# Toon de grafiek
plot_gegevens(afstanden, tijden, 'Drop data zoomed in')

### De kleinste kwadraten methode

Om een model te fitten op onze gegevens maken we gebruik van de methode van de kleinste kwadraten. Deze zal op zoek gaan naar een functie die zo dicht mogelijk bij onze gegevens ligt. 
Voor we de methode kunnen toepassen, moeten we eerst een idee hebben van het soort functie die we gaan fitten op onze gegevens. Wanneer je de grafiek van de valbeweging bekijkt, zou je moeten zien dat deze lijkt op een parabool. Uit de wiskunde weten we dat de grafiek van de parabool volgens het volgende voorschrift verloopt:

\begin{equation}
y = ax^2 + bx + c
\end{equation}

We kennen heel wat waarden voor $x$ en $y$. Dit zijn de metingen die we gedaan hebben met onze sensor. De waarden van $a$, $b$ en $c$ kennen we echter niet. We kunnen de methode van de kleinste kwadraten gebruiken om deze te bepalen. De methode van de kleinste kwadraten zal een stelsel van vergelijkingen oplossen om zo de waarden van $a$, $b$ en $c$ te bepalen. Dit stelsel ziet er als volgt uit:

\begin{equation}
\begin{cases}
y_1 = a x_1^2 + b x_1 + c \\
y_2 = a x_2^2 + b x_2 + c \\
\vdots \\
y_n = a x_n^2 + b x_n + c \\
\end{cases}
\end{equation}

Waarbij de koppels $(x_n, y_n)$ het tijdstip en de hoogte van de n de meting voorstellen (tijdstip_n, hoogte_n). Merk op dat dit stelsel meer vergelijkingen heeft dan onbekenden. We zoeken dus niet zomaar een oplossing maar we zoeken een oplossing waarvoor de som van de kwadraten van de fouten zo klein mogelijk is. Concreet zullen we een $a$, $b$ en $c$ kiezen waarvoor

\begin{equation}
\epsilon = \sum_{1}^{n}{(hoogte_n - y_n)^2}
\end{equation}

minimaal is.

We kunnen het stelsel van vergelijkingen voorstellen in matrixvorm:

\begin{equation}
y = X \beta
\end{equation}

met

\begin{equation}
y = \begin{pmatrix}
  y_1 \\
  y_2 \\
  \vdots \\
  y_n
\end{pmatrix}
\end{equation}

\begin{equation}
X = \begin{pmatrix}
  x_1^2 & x_1 & 1 \\
  x_2^2 & x_2 & 1 \\
  \vdots & \vdots & \vdots \\
  x_n^2 & x_n & 1
\end{pmatrix}
\end{equation}

\begin{equation}
\beta = 
\begin{pmatrix}
  a \\
  b \\
  c
\end{pmatrix}
\end{equation}


Wiskundigen hebben aangetoond dat je de optimale waarde voor $\beta$ kan vinden aan de hand van de volgende berekening:

\begin{equation}
\beta = (X^T X)^{-1} X^T y
\end{equation}

Om de berekining uit te kunnen voeren moeten we dus de matrix $X$ en de vector $y$ opstellen. De berekening kunnen we Python laten uitvoeren

We beginnen met de tijden en de afstanden voor te stellen als vectoren.

In [ ]:
x = tijden
y = afstanden.T[0]

print(x)
print(y)

Nu kunnen we de matrix $X$ opstellen door als eerste kolom het kwadraat van de vector $x$ te nemen, als tweede kolom de vector $x$ zelf en als laatste kolom een vector met enkel het getal 1.

In [ ]:
X = np.vstack((x**2, x, np.ones_like(x))).T
print(X)

Nu kunnen we Python gebruiken om de berekening uit te voeren.

In [ ]:
coef = np.linalg.inv(X.T @ X) @ X.T @ y
print(coef)

Zo nu hebben we de waarden van de coëficienten van de tweedegraadsvergelijking ($a$, $b$ en $c$). We kunnen deze vergelijking nu ook plotten en kijken wat het verschil is met de echte data.

In [ ]:
# Plot de vergelijking met de nieuwe coëfficiënten
x = np.linspace(0, 0.4, 100)
y = coef[0] * x**2 + coef[1] * x + coef[2]

plt.figure()
plt.plot(x, y)
plt.xlabel('Tijd (s)')
plt.ylabel('Hoogte (m)')
plt.title('Model van de valbeweging vergeleken met de gemeten data')
plt.plot(tijden, afstanden)
plt.legend(['Wiskundig model', 'Gemeten data'])
plt.show()




Zoals je op de grafiek ziet liggen het model en de echte waarden zeer dicht bij elkaar. We lijken dus een goed model te hebben voor onze valbeweging. 

## Analyse van de beweging

Op basis van het model kunnen we proberen om de beweging beter te begrijpen. We kunnen bijvoorbeeld de beginhoogte van het object berekenen:

In [ ]:
# Beginhoogte volgens het wiskundig model
beginhoogte = coef[0] * 0**2 + coef[1] * 0 + coef[2]
print(f'Beginhoogte model: {beginhoogte} cm')
print(f'Beginhoogte data: {afstanden[0]} cm')

Op basis van ons model kunnen we ook het verloop van de snelheid analyseren. De snelheid is de afgeleide van de positie van het object. Door de afgeleide te nemen van de functie:

\begin{equation}
y = ax^2 + bx + c
\end{equation}

Krijgen we de vergelijking

\begin{equation}
y' = 2ax + b
\end{equation}

De snelheid verloopt dus volgens de grafiek van een rechte. Dat kunnen we ook visualiseren.

In [ ]:
snelheden = 2 * coef[0] * tijden + coef[1]
plot_gegevens(snelheden.reshape(-1, 1), tijden, 'Snelheid', xlabel='Tijd (s)', ylabel='Snelheid (m/s)')

We merken dat de snelheid steeds negatiever wordt. Dit wil dus zeggen dat hoe langer het object valt, hoe sneller het zal gaan. Tot het de grond raakt natuurlijk.

We kunnen dezelfde redenering toepassen om ook de versnelling te bepalen. 

\begin{equation}
y' = 2ax + b
\end{equation}

Nemen we de afgeleide van de functie voor de snelheid, dan krijgen we de vergelijking:

\begin{equation}
y'' = 2a
\end{equation}

De versnelling is dus een constante.

In [ ]:
versnellingen = 2 * coef[0] * np.ones_like(tijden)	
plot_gegevens(versnellingen.reshape(-1, 1), tijden, 'Versnelling', xlabel='Tijd (s)', ylabel='Versnelling (m/s^2)')

We zien dat de versnelling van het object 8,22 m/s² is. Dat is niet exact de valversnelling die we in boeken kunnen lezen (9,81 m/s²) maar ligt er ook niet ver van. De versnelling is hier negatief omdat we in ons referentiekader ervoor gekozen hebben om de positieve z-as naar boven te doen wijzen.

# Opdracht

We hebben nu een wiskundig model van onze valbeweging. Dit model kunnen we nu gebruiken om voorspellingen te doen. 

Gebruik het wiskundig model om te voorspellen hoe lang het voorwerp zal vallen als we het vanop een hoogte van 2m laten vallen.

# Opdracht

Je kan nog andere bewegingen vastleggen aan de hand van een afstandssensor. Denk bijvoorbeeld aan een gewicht dat aan een veer hangt of een wagentje dat op een horizontaal oppervlak rijdt. Bedenk een andere beweging waarvan je het verloop kan registreren aan de hand van de sonar sensor. Registreer dat verloop en probeer er een wiskundig model voor te bedenken.